<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github//jayralencar/pictogram-to-sense/blob/main/Download_Dataset.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/jayralencar/pictogram-to-sense/blob/main/Download_Dataset.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://colab.research.google.com/drive/1wLgg4CtmR1ZJ2bMTWN5dcsamw8ZOAdN0"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# Dowload ARASAAC

The aim of this Notebook is to download the pictograms from [ARASAAC](http://arasaac.org/) to be used as a dataset for fine-tuning CLIP.

In our case, fine-tuning CLIP consists of adjusting its parameters using a pictogram dataset. 
For this, we need a dataset of pictograms paired with their textual description.
In addition to keywords, ARASAAC has a meaning text for some pictograms. This way, we used the pictograms from ARASAAC and their descriptions to make (image, text) pairs and used them as a dataset. For this, we used the ARASAAC developers API to download the image and description of all the available pictograms. When the pictogram has two or more descriptions, we copy the image as many times as necessary and pair each copy with its respective description. After this procedure, we acquired 22,448 (image, text) pairs, which we split in 80\% for training and 20\% for validation. 

Download the dataset [here](https://drive.google.com/file/d/1P627Qusf8QmaVvrEoevWJNjuUp2luOyd/view), or execute the cells below.

In [ ]:
% mkdir data

In [ ]:
import urllib
import requests

def download_image(pictogram_id, names=[],details=None):
  if details is None:
    url = "https://api.arasaac.org/api/pictograms/"+str(pictogram_id)+"?download=true"
    filename = 'data/'+str(pictogram_id)+".png"
  else:
    details['download']='true'
    url = "https://api.arasaac.org/api/pictograms/{0}?".format(pictogram_id)+urllib.parse.urlencode(details)
    print(url)
    filename = 'data/{0}_{1}_{2}.png'.format(pictogram_id,details['skin'],details['hair'])
  r = requests.get(url, allow_redirects=True)
  for filename in names:
    open("data/"+filename+'.png', 'wb').write(r.content)
# download_image(4665,names=['a','b'])

In [ ]:
r = requests.get("https://api.arasaac.org/api/pictograms/all/en")

In [ ]:
import string
from tqdm import tqdm
for pictogram in tqdm(r.json()):
  definition = None
  names = []
  i = 1
  for kw in pictogram['keywords']:
    if 'meaning' in kw:
      definition = "{0}: {1}".format(kw['keyword'],kw['meaning'])
    else:
      definition = kw['keyword']
    open("./data/{0}_{1}.txt".format(pictogram['_id'],i),'w').write(definition)
    names.append("{0}_{1}".format(pictogram['_id'],i))
    i+=1
  download_image(pictogram['_id'],names)

100%|██████████| 11779/11779 [1:21:04<00:00,  2.42it/s]


In [ ]:
!zip data.zip -r ./data